## Function Caching (Memoization)

##### Fibonacci numbers

In [128]:
import time

In [129]:
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [130]:
start = time.time()
print(fib(36))
print("소요시간: %s 초" % str(round(time.time() - start, 2)))

14930352
소요시간: 4.34 초


##### 시간 및 계산 비용이 크다
- 재귀호출을 2^n - 1번 한다.
- 숫자가 커질수록 중복 계산횟수가 기하급수적으로 늘어난다.

In [131]:
# fib(36)

(2 ** 36) - 1

68719476735

##### 재귀호출은 (2^n)-1 번 수행하지만, 실제 필요한 함수의 결과값은 n개 뿐이다.
- 한 번 계산한 결과를 저장하고
- 저장한 값을 꺼내 쓰는 방식으로 바꿔보자

In [132]:
cache = {}

def fib(n):
    if n in cache:
        return cache[n]
    
    if n < 2:
        return n
    
    cache[n] = fib(n-1) + fib(n-2)
    return cache[n]

In [133]:
fib(36)

14930352

In [134]:
cache

{2: 1,
 3: 2,
 4: 3,
 5: 5,
 6: 8,
 7: 13,
 8: 21,
 9: 34,
 10: 55,
 11: 89,
 12: 144,
 13: 233,
 14: 377,
 15: 610,
 16: 987,
 17: 1597,
 18: 2584,
 19: 4181,
 20: 6765,
 21: 10946,
 22: 17711,
 23: 28657,
 24: 46368,
 25: 75025,
 26: 121393,
 27: 196418,
 28: 317811,
 29: 514229,
 30: 832040,
 31: 1346269,
 32: 2178309,
 33: 3524578,
 34: 5702887,
 35: 9227465,
 36: 14930352}

##### Memoization
- 컴퓨터 프로그램이 동일한 계산을 반복해야 할 때, 이전에 계산한 값을 메모리에 저장함으로써 동일한 계산의 반복 수행을 제거하여 프로그램 실행 속도를 빠르게 하는 기술.

##### 좀 더 쉽게 쓸 수 있도록 데코레이터로 만들어보자

In [135]:
def memoize(f):
    cache = {}
    
    def wrapper(n):
        if n in cache:
            return cache[n]
        
        cache[n] = f(n)
        
        return cache[n]
    
    return wrapper

In [136]:
@memoize
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [137]:
fib(35)

9227465

- 함수는 1개의 인자만 받는다(n)
- 그 인자는 사전의 키가 될 수 있는 객체여야 한다

##### 개선된 데코레이터

In [138]:
def memoize(f):
    cache = {}
    
    def wrapper(*args, **kwargs):
        key = str(args) + str(kwargs)
        
        if key in cache:
            return cache[key]
        
        cache[key] = f(*args, **kwargs)
        #print(cache)
        
        return cache[key]
    
    return wrapper

In [139]:
@memoize
def add(a, b):
    return a + b

In [140]:
add(4, 5)

{'(4, 5){}': 9}


9

##### 파이썬 3.2+부터 lru_cache 데코레이터를 사용해서 쉽게 함수의 반환값들을 캐싱할 수 있다.
- lru_cache 데코레이터는 주어진 함수를 더 빨리 작동하도록 해준다.
- LRU(least recently used) 알고리즘으로 동작.

In [141]:
from functools import lru_cache

In [142]:
@lru_cache(maxsize=128)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [143]:
fib(35)

9227465

##### lru_cache를 이용하여 동일한 함수를 반복 수행하면 그냥 캐시를 읽는 것 밖에 안되므로 0에 가까워짐

In [144]:
start = time.time()
print(fib(500))
print("소요시간: %s 초" % str(round(time.time() - start, 10)))

139423224561697880139724382870407283950070256587697307264108962948325571622863290691557658876222521294125
소요시간: 0.005302906 초


In [146]:
start = time.time()
print(fib(500))
print("소요시간: %s 초" % str(round(time.time() - start, 10)))

139423224561697880139724382870407283950070256587697307264108962948325571622863290691557658876222521294125
소요시간: 0.0013701916 초


##### 캐시 지우기

In [147]:
fib.cache_clear()

#### 주의사항
- 캐시에 저장한 값을 다시 꺼내쓰는 일이 적다면, 성능향상보다 캐시를 유지하는 데 드는 비용이 더 클 수 있다.
- float의 부동소수점이 가지는 오차때문에 memoization 적용으로 인한 유익이 없을 수도 있다.